In [1]:
import torch
import numpy as np
from collections import OrderedDict
from graph.matrixview import render_incidence
from engine.torchengine import AnalyticalSetSympy, EliminateAnalysisMergeResiduals, EliminateAnalysis, FunctionSympy,get_analysis_structure, ElimResidual, ParallelResiduals, ParallelAnalysis, ipoptsolvercon
from engine.torchdata import load_file, process_expression, load_vals, generate_optim_functions, print_formatted_table, perturb
from engine.loaddata import process_json
from functools import partial
from collections import namedtuple
# Set the print options
np.set_printoptions(formatter={'float': lambda x: "{:0.2f}".format(x).rstrip('0').rstrip('.')})

In [2]:
data = load_file('testproblems')
problem_id = 2
_, fdata = list(data.items())[problem_id]
polynomials, indices, edges, objective = process_json(fdata)
idxrev = {var.item():key for key,var in indices.items()}

sets = {idx:AnalyticalSetSympy(poly, indices=indices).reassign(edges[1][idx][0],  rational=True) for idx,poly in polynomials.items()}
objective = FunctionSympy(objective, indices)

## Different ways to define orders

### A) Individual

In [3]:
analyses = {key: s.analysis for key,s in sets.items()}
residuals = {key: s.residual for key,s in sets.items()}

### 1) Pure

In [4]:
order = list(sorted(analyses.keys()))
A, B = [], order

### Option 1

In [5]:
# Merge residuals only
solvefor = torch.tensor([])
R = EliminateAnalysisMergeResiduals(functions=[residuals[idx] for idx in B])
P = EliminateAnalysis([analyses[idx] for idx in A], [objective,R])
objective_idx,residual_idx,equality_idx,inequality_idx = 0,1,None,None

### Option 3

In [17]:
solvefor = torch.tensor([])
T = ParallelResiduals([sets[idx].analysis for idx in B], [objective])
P = EliminateAnalysis([sets[idx].analysis for idx in A], [T], flatten=True)
objective_idx,residual_idx,equality_idx,inequality_idx = 0,None,1,None

### Initial guess

In [6]:
problem_id = 2 
n = len(indices)
torch.manual_seed(2)
xs = torch.rand(n, dtype=torch.float64)

In [7]:
optim_indices = P.structure[0]

In [8]:
x0 = perturb(xs, 0, optim_indices, seed=12)
print_formatted_table([xs, x0], indices, idxrev, subset=optim_indices)

x_0   x_10  x_11  x_13  x_2   x_3  x_9   x_4   x_5   x_6  x_8   x_1   x_14  x_7   x_12
0.918 0.082 0.683 0.808 0.709 0.54 0.462 0.913 0.526 0.12 0.693 0.097 0.733 0.267 0.62
0.918 0.082 0.683 0.808 0.709 0.54 0.462 0.913 0.526 0.12 0.693 0.097 0.733 0.267 0.62


### Optimization

In [9]:
# MAKE SURE TO SET INEQUALITY DIRECTION, it is different for scipy and ipopt
xguess, obj_function, ineq_function, eq_function, dobj, dineq, deq, hobj = generate_optim_functions(P,
        optim_indices, x0, inequality_direction='negative-null', 
        objective=objective_idx, residuals=residual_idx, equalities=equality_idx)
bnds_problem = [(None, None) for elt in optim_indices]

In [10]:
eq_function(xguess).numpy(), ineq_function(xguess).numpy(), obj_function(xguess)

(array([-0.34, 0.27, 1.2, -0.9, 0.7, 0.08, 0.96, 0.93, -0.17, -0.08]),
 array([], dtype=float32),
 1.0715371477653555)

# IPOPT

In [11]:
xsol, info,storeiter,elapsed_time = ipoptsolvercon(xguess, obj_function, ineq_function, eq_function, dobj, dineq, deq, bnds_problem)

In [12]:
obj_function(xsol)

0.9691691803346383

In [13]:
objective.expression

(x_0 - 1)**2 + (x_10 - 1)**2 + (x_11 - 1)**2 + (x_13 - 1)**2 + (x_2 - 1)**2

In [14]:
storeiter

[100]

In [15]:
print_formatted_table([x0], indices, idxrev, subset=optim_indices)

x_0   x_10  x_11  x_13  x_2 x_3   x_9 x_4   x_5  x_6   x_8    x_1    x_14  x_7   x_12 
1.003 0.016 1.015 0.997 1   1.667 0.6 5.333 -0.4 0.153 -0.334 13.965 2.104 2.377 3.258


In [16]:
[(sets[elt].outputvar, sets[elt].expression) for elt in B]

[(x_3, 5/3),
 (x_9, 3/(5*x_2)),
 (x_8, (-x_5*x_6 - 9)/(5*x_0*x_4)),
 (x_6, 9/(7*x_1*x_9)),
 (x_1, 7/(6*x_10*x_4)),
 (x_7, (3*x_1*x_11 + 3*x_6 + 2)/(9*x_14)),
 (x_14, 5/x_7),
 (x_5, -2/5),
 (x_12, 1/(2*x_6)),
 (x_4, -4*x_13*x_8 + 4)]

In [17]:
print_formatted_table([sets[9].analysis(x0).detach().numpy()], indices, idxrev, subset=optim_indices)

x_0   x_10  x_11  x_13  x_2 x_3   x_9 x_4   x_5  x_6   x_8    x_1    x_14  x_7   x_12 
1.003 0.016 1.015 0.997 1   1.667 0.6 5.333 -0.4 0.153 -0.334 13.965 2.104 2.377 3.258


### Save results

In [82]:
import json

In [ ]:
name = 'results.json'
json_string = json.dumps(results, indent=4)
with open('../applications/data/{}'.format(name), 'w') as file:
    file.write(json_string)
